In [73]:
import requests
import csv
import re
import random
import pandas as pd
import time
from bs4 import BeautifulSoup
import os
import numpy as np
from numpy import arange
import string
from requests.exceptions import ProxyError

In [74]:
#list of user agents to cycle through
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [75]:
#proxy list for scraper
proxy_list = ['104.248.232.204:3128','142.93.15.115:3128','68.183.114.87:3128',
              '35.245.197.159:3128','208.180.237.55:31012','35.221.50.231:3128',
               '176.105.100.62:3128','188.152.173.118:8118','2.39.7.150:8118',
               '188.216.26.67:8118','104.248.51.47:8080','103.99.177.247:80'
             ]

In [76]:
inexp_page1 = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Sydney%2C%20New%20South%20Wales&l=g%3A151.36174709619138%2C-33.557858573833116%2C150.50344021142575%2C-34.128158965023005&attrs=RestaurantsPriceRange2.1'

In [77]:
mod_page1 = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Sydney%2C%20New%20South%20Wales&l=g%3A151.36174709619138%2C-33.557858573833116%2C150.50344021142575%2C-34.128158965023005&attrs=RestaurantsPriceRange2.2&ed_attrs=RestaurantsPriceRange2.1'

In [78]:
pricey_page1 = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Sydney%2C%20New%20South%20Wales&l=g%3A151.36174709619138%2C-33.557858573833116%2C150.50344021142575%2C-34.128158965023005&attrs=RestaurantsPriceRange2.3&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2'

In [79]:
highend_page1 = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Sydney%2C%20New%20South%20Wales&l=g%3A151.36174709619138%2C-33.557858573833116%2C150.50344021142575%2C-34.128158965023005&attrs=RestaurantsPriceRange2.4&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2%2CRestaurantsPriceRange2.3'

***Pages of reviews for each resturant*** <br>
1) We can use all characters between "datePublished.....and... /iv>" to capture the actual texts of reviews <br>
2) We can also use "ratingValue" content=" as a marker for star rating <br>
3) we will have to use the total number of reviews (found in plain html on the first page of all resturants) to calculate how many pages of reviews (assuming 20 reviews per page) 

***Pages of resturants for each price point***<br>
1) We can manually scrape total pages and total number of resturants and then we assume 30 resturants per page <br>
2) We will have to also scrape all resturant URLs for individual resturants for pre-review scraping <br>
3) We can use "ranking"....."name"......"neighbohoods" as markers for each resturant name we have to scrape and process

***Scrapper Pseudo Code***<br>
1- Calculate the number of pages of resturants each price point has <br>
2- Extract URLS and process all resturant names for each price point <br>
3- Calculate the number of pages of reviews each resturant has <br>
4- Use numbers from above to establish dataframes and loops <br>
5- Finally scrape all reviews and star ratings into 4 seperate dataframes, one for each price point </br>

In [26]:
#Put price point URL first pages in list
price_points = list()
price_points.append(inexp_page1)
price_points.append(mod_page1)
price_points.append(pricey_page1)
price_points.append(highend_page1)

***These are manually copy and pasted from the website***, these are the number of resturants per price point

In [27]:
rest_numbers = {'inexp':1522,'mod':2982,'pricey':495,'high-end':98}
#for a total of 5097 resturants across sydney

In [28]:
page_numbers = {'inexp':51,'mod':100,'pricey':17,'high-end':4}
#for a total of 172 pages of resturants

## 2.0 Extract and Process all resturant URLs for each price point
### 2.1 Cheapest price point (inexp)
#### First we construct all URLs for each page of resturants
After examining the URLs,***'&start=30'*** is added to the end of the url for page 2 (30 is added for every page)
The last page url of any price point is therefore constructed by adding ***'&start=(MaxPages-1)*30'***, so
***page2 = inexp_page1 + '&start=30'***

In [80]:
#first define a function to handle all scraping of dirty URLS
def getdirty_urls(page1,numofpages):
    #lets now save this as resturant count, page 2 = 30, page 3 = 60....page 15 = 420
    resturant_count = list()
    #construct count to add to URL
    for x in range(1,numofpages):
        resturant_count.append(str((x*30)))
    #Get resturant names from each page, start with page 1 then loop through the rest
    dirty_links = list()
    try:
        response = requests.get(page1,headers={'user-agent': random.choice(user_agent_list)}, timeout=5,proxies={'https':random.choice(proxy_list)})
    except ProxyError:
        print('PROXY ERROR WITH PAGE: 1. retrying with new proxy')
        time.sleep(10)
        #response = requests.get(page1,headers={'user-agent': random.choice(user_agent_list)}, timeout=5,proxies=random.choice(proxy_list))   
        pass
    except requests.exceptions.Timeout:
        print('TIMEOUT ERROR WITH PAGE: ', int(int(count)/30-1) ,'. sleeping for 10seconds then retrying')
        time.sleep(10)
                #response = requests.get(page1,headers={'user-agent': random.choice(user_agent_list)}, timeout=5,proxies=random.choice(proxy_list))
        pass
    content = response.content
    copycontent = str(content)
    dirtylinks = list()
    dirtylinks = re.findall('href="/biz/(.*?)" t',copycontent)
    dirty_links.append(dirtylinks)
    for count in resturant_count:
        time.sleep(10)
        new_page = page1 + '&start=' + count
        try:
                response = requests.get(page1,headers={'user-agent': random.choice(user_agent_list)}, timeout=5,proxies={'https':random.choice(proxy_list)})
        except ProxyError:
                print('PROXY ERROR WITH PAGE: ', int(int(count)/30-1) ,'. retrying with new proxy')
                time.sleep(10)
                #response = requests.get(page1,headers={'user-agent': random.choice(user_agent_list)}, timeout=5,proxies=random.choice(proxy_list))
                pass
        except requests.exceptions.Timeout:
                print('TIMEOUT ERROR WITH PAGE: ', int(int(count)/30-1) ,'. sleeping for 10seconds then retrying')
                time.sleep(10)
                #response = requests.get(page1,headers={'user-agent': random.choice(user_agent_list)}, timeout=5,proxies=random.choice(proxy_list))
                pass
        content = response.content
        copycontent = str(content)
        dirtylinks = list()
        dirtylinks = re.findall('href="/biz/(.*?)" t',copycontent)
        dirty_links.append(dirtylinks)
        print('Completed Page : ',int(int(count)/30-1))
    return dirty_links

In [104]:
inexp_dirty_links = getdirty_urls(inexp_page1,51)

Completed Page :  0
Completed Page :  1
Completed Page :  2
TIMEOUT ERROR WITH PAGE:  3 . sleeping for 10seconds then retrying
Completed Page :  3
PROXY ERROR WITH PAGE:  4 . retrying with new proxy
Completed Page :  4
TIMEOUT ERROR WITH PAGE:  5 . sleeping for 10seconds then retrying
Completed Page :  5
Completed Page :  6
TIMEOUT ERROR WITH PAGE:  7 . sleeping for 10seconds then retrying
Completed Page :  7
Completed Page :  8
Completed Page :  9
Completed Page :  10
Completed Page :  11
Completed Page :  12
Completed Page :  13
PROXY ERROR WITH PAGE:  14 . retrying with new proxy
Completed Page :  14
PROXY ERROR WITH PAGE:  15 . retrying with new proxy
Completed Page :  15
Completed Page :  16
Completed Page :  17
PROXY ERROR WITH PAGE:  18 . retrying with new proxy
Completed Page :  18
TIMEOUT ERROR WITH PAGE:  19 . sleeping for 10seconds then retrying
Completed Page :  19
PROXY ERROR WITH PAGE:  20 . retrying with new proxy
Completed Page :  20
TIMEOUT ERROR WITH PAGE:  21 . sleep

In [69]:
sumi = 0
for setlinks in inexp_dirty_links:
    sumi = sumi+ len(setlinks)
print(sumi)

1729


There are actually 1522 resturants in the inexpensive price point, that fact that we've scraped more than this is a good sign.

In [71]:
np.savetxt("dirty_inexp_urls.csv", inexp_dirty_links, delimiter=",", fmt='%s', header='Resturant Names')

### 2.2 Moderate price point(mod)

In [86]:
mod_dirty_links = getdirty_urls(mod_page1,100)

TIMEOUT ERROR WITH PAGE:  0 . sleeping for 10seconds then retrying
Completed Page :  0
PROXY ERROR WITH PAGE:  1 . retrying with new proxy
Completed Page :  1
Completed Page :  2
Completed Page :  3
TIMEOUT ERROR WITH PAGE:  4 . sleeping for 10seconds then retrying
Completed Page :  4
TIMEOUT ERROR WITH PAGE:  5 . sleeping for 10seconds then retrying
Completed Page :  5
TIMEOUT ERROR WITH PAGE:  6 . sleeping for 10seconds then retrying
Completed Page :  6
PROXY ERROR WITH PAGE:  7 . retrying with new proxy
Completed Page :  7
TIMEOUT ERROR WITH PAGE:  8 . sleeping for 10seconds then retrying
Completed Page :  8
PROXY ERROR WITH PAGE:  9 . retrying with new proxy
Completed Page :  9
Completed Page :  10
Completed Page :  11
PROXY ERROR WITH PAGE:  12 . retrying with new proxy
Completed Page :  12
TIMEOUT ERROR WITH PAGE:  13 . sleeping for 10seconds then retrying
Completed Page :  13
PROXY ERROR WITH PAGE:  14 . retrying with new proxy
Completed Page :  14
PROXY ERROR WITH PAGE:  15 . r

In [87]:
np.savetxt("dirty_mod_urls.csv", mod_dirty_links, delimiter=",", fmt='%s', header='Resturant Names')

### 2.3 Pricey price point(pricey)

In [88]:
pricey_dirty_links = getdirty_urls(pricey_page1,17)

TIMEOUT ERROR WITH PAGE:  0 . sleeping for 10seconds then retrying
Completed Page :  0
TIMEOUT ERROR WITH PAGE:  1 . sleeping for 10seconds then retrying
Completed Page :  1
TIMEOUT ERROR WITH PAGE:  2 . sleeping for 10seconds then retrying
Completed Page :  2
TIMEOUT ERROR WITH PAGE:  3 . sleeping for 10seconds then retrying
Completed Page :  3
PROXY ERROR WITH PAGE:  4 . retrying with new proxy
Completed Page :  4
Completed Page :  5
TIMEOUT ERROR WITH PAGE:  6 . sleeping for 10seconds then retrying
Completed Page :  6
PROXY ERROR WITH PAGE:  7 . retrying with new proxy
Completed Page :  7
TIMEOUT ERROR WITH PAGE:  8 . sleeping for 10seconds then retrying
Completed Page :  8
Completed Page :  9
PROXY ERROR WITH PAGE:  10 . retrying with new proxy
Completed Page :  10
TIMEOUT ERROR WITH PAGE:  11 . sleeping for 10seconds then retrying
Completed Page :  11
Completed Page :  12
TIMEOUT ERROR WITH PAGE:  13 . sleeping for 10seconds then retrying
Completed Page :  13
Completed Page :  14


In [102]:
#np.savetxt("dirty_pricey_urls.csv", pricey_dirty_links[6], newline=",", fmt='%s', header='Resturant Names')

### 2.4 High-end price point (highend)

In [108]:
highend_dirty_links = getdirty_urls(highend_page1,4)

Completed Page :  0
Completed Page :  1
TIMEOUT ERROR WITH PAGE:  2 . sleeping for 10seconds then retrying
Completed Page :  2


In [109]:
highend_dirty_links

[[],
 [],
 ['sixpenny-stanmore',
  'sixpenny-stanmore',
  'sixpenny-stanmore?hrid=BiGZfj97GNk0esD0xlNZEw',
  'flower-drum-h-hinchinbrook',
  'flower-drum-h-hinchinbrook',
  'flower-drum-h-hinchinbrook?hrid=lN1O8bW_ltr5Imt5E1OBwg',
  'mj%C3%B8lner-sydney',
  'mj%C3%B8lner-sydney',
  'mj%C3%B8lner-sydney?hrid=VqYFIPBjkijiLBen3tjByg',
  'sails-on-lavender-bay-mcmahons-point-2',
  'sails-on-lavender-bay-mcmahons-point-2',
  'sails-on-lavender-bay-mcmahons-point-2?hrid=PvDdGVsKeOFl-svsWbLrbw',
  'automata-chippendale',
  'automata-chippendale',
  'kensington-street-chippendale">Kensington Street</a></div></div><div class="lemon--div__373c0__1mboc u-space-b1 border-color--default__373c0__2oFDT"><div class="lemon--div__373c0__1mboc border-color--default__373c0__2oFDT">Chippendale</div></div></div></div></div><div class="lemon--div__373c0__1mboc u-space-t2 u-space-b2 border-color--default__373c0__2oFDT"><div class="lemon--div__373c0__1mboc arrange__373c0__UHqhV vertical-align-middle__373c0__2T

In [221]:
inexp_clean_links = list()
for linkset in dirty_links:
    links_set = set(linkset)
    for link in links_set:
        if 'hrid' in link:
            link = 'https://www.yelp.com/biz/'+link
            inexp_clean_links.append(link)

Some links need an extra round of cleaning:

In [231]:
num = 0
linkindex = list()
for link in inexp_clean_links:
    num = num + 1
    if len(link) > 500:
        linkindex.append(num)

In [240]:
for index in linkindex:
    dirtylinks = list()
    dirtylinks = re.findall('href="/biz/(.*?)" ',str(inexp_clean_links[index]))
    inexp_clean_links[index] = dirtylinks
    print(dirtylinks)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [70]:
#the captured strings need to be cleaned once more to extract only the resturant names:
clean_names = list()
for name in dirty_names:
    for string in name:
        cleanstring = re.findall('name="(.*?)"',string)
        clean_names.append(cleanstring)

In [72]:
#we then need to convert the resturant names to strings
final_names = list()
for name in clean_names:
    name = str(name)
    name = name[2:]
    final_names.append(str(name))
    #print(name)